This notebook is for relaxting an initialized system of randomly placed polymer chains. This is accomplished by slowly ramping up a soft potential [DPD](https://hoomd-blue.readthedocs.io/en/stable/module-md-pair.html#hoomd.md.pair.dpd) such that the repulsion between particles matches that of a WCA repulsion between monomers. The system is then switched over to [WCA potential](https://hoomd-blue.readthedocs.io/en/stable/module-md-pair.html#hoomd.md.pair.lj). Ramping from DPD to WCA is facilitated by a harmonic bonding potential to decrease the stiffness of the system. After switching to a WCA potential, the [harmonic bond potential](https://hoomd-blue.readthedocs.io/en/stable/module-md-bond.html#hoomd.md.bond.harmonic) is converted to a [FENE bond](https://hoomd-blue.readthedocs.io/en/stable/module-md-bond.html#hoomd.md.bond.fene) to finally finally obtain the Kremer-Grest bead-spring chain model. 

In [1]:
import hoomd
from hoomd import *
import numpy as np
from hoomd import md

First we initialize hoomd and set up our system parameters and the parameters for each of our potentials. 

In [ ]:
# Initialize context
hoomd.context.initialize("--mode=cpu");
system = hoomd.init.read_gsd('init.gsd')

# Cell list
nl = md.nlist.cell();

# Get box size
Lx = system.box.Lx;

# Temperature
T_run = 1.0

# Define time steps for running
n_init = 1E4
n_ramp = 1E6
A_init = 1.0
A_final = 400.0
A_ramp = np.linspace(A_init,A_final,n_ramp)

# Define bead potential
eps_lj = 1.0
sigma = 1.0
rcut = 2.0**(1.0/6.0)*sigma

In [ ]:

# Alternate bonding potential
dpd_potential = md.pair.dpd(r_cut=rcut, nlist=nl, kT=1.0, seed=0)
dpd_potential.pair_coeff.set('A', 'A', A=1.0, gamma = 0.0)

# Define bond potential
bonding = md.bond.harmonic();
bonding.bond_coeff.set('polymer', k=30.0, r0=1.5)

# Groups
all = group.all();

# Set up integrator
md.integrate.mode_standard(dt=0.001);
integrator = md.integrate.nvt(group=all, tau=1.0, kT=T_run)
integrator.randomize_velocities(seed=42)

# Period for dump
file_name = 'test_run'
dump_steps = 1000#500
gsd = dump.gsd(filename=file_name+'.gsd', period=dump_steps, group=all, overwrite=True)

run(n_init)

# Running ramping of A
for i in range(0,len(A_ramp)):
    dpd_potential.pair_coeff.set('A', 'A', A=A_ramp[i], gamma = 0.0)
    print(A_ramp[i])
    run(1E2,quiet=True)


dpd_potential.disable()

In [ ]:
lj = md.pair.lj(r_cut=rcut, nlist=nl);
lj.pair_coeff.set('A', 'A', epsilon=eps_lj, sigma=sigma)
lj.set_params(mode="shift")

run(1E5)

bonding.disable()
bonding = md.bond.fene();
bonding.bond_coeff.set('polymer', k=30.0, r0=2.0*sigma, sigma=sigma, epsilon=1.0)

run(1E6)

-----
You are using DPD. Please cite the following:
* C L Phillips, J A Anderson, and S C Glotzer. "Pseudo-random number generation
  for Brownian Dynamics and Dissipative Particle Dynamics simulations on GPU
  devices", Journal of Computational Physics 230 (2011) 7191--7201
-----
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 1 exclusions             : 20
notice(2): Particles with 2 exclusions             : 980
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:08 | Step 10000 / 10000 | TPS 1330.36 | ETA 00:00:00
Average TPS: 1330.04
---------
-- Neighborlist stats:
162 normal updates / 100 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 36 / n_neigh_avg: 11.715
shortest rebuild period: 39
-- Cell list stats:
Dimension: 5, 5, 5
n_min    : 1 / n_max: 16 / n_avg: 8
** run complete **
1.0
1.0003990003990004
1.0007980007980009
1.0011970011970013
1.0

1.1548121548121548
1.1552111552111552
1.1556101556101557
1.156009156009156
1.1564081564081565
1.1568071568071567
1.1572061572061572
1.1576051576051576
1.158004158004158
1.1584031584031584
1.1588021588021589
1.1592011592011593
1.1596001596001595
1.15999915999916
1.1603981603981603
1.1607971607971608
1.1611961611961612
1.1615951615951616
1.161994161994162
1.1623931623931625
1.162792162792163
1.1631911631911631
1.1635901635901635
1.163989163989164
1.1643881643881644
1.1647871647871648
1.1651861651861652
1.1655851655851657
1.1659841659841659
1.1663831663831663
1.1667821667821667
1.1671811671811672
1.1675801675801676
1.167979167979168
1.1683781683781684
1.1687771687771689
1.1691761691761693
1.1695751695751695
1.16997416997417
1.1703731703731703
1.1707721707721708
1.1711711711711712
1.1715701715701716
1.171969171969172
1.1723681723681723
1.1727671727671727
1.173166173166173
1.1735651735651735
1.173964173964174
1.1743631743631744
1.1747621747621748
1.1751611751611752
1.1755601755601757
1.1759

1.3295743295743296
1.32997332997333
1.3303723303723305
1.3307713307713307
1.3311703311703311
1.3315693315693315
1.331968331968332
1.3323673323673324
1.3327663327663328
1.3331653331653333
1.3335643335643335
1.333963333963334
1.3343623343623343
1.3347613347613347
1.3351603351603352
1.3355593355593356
1.335958335958336
1.3363573363573364
1.3367563367563369
1.337155337155337
1.3375543375543375
1.337953337953338
1.3383523383523384
1.3387513387513388
1.3391503391503392
1.3395493395493396
1.3399483399483398
1.3403473403473405
1.3407463407463407
1.3411453411453411
1.3415443415443415
1.341943341943342
1.3423423423423424
1.3427413427413428
1.3431403431403433
1.3435393435393435
1.3439383439383439
1.3443373443373443
1.3447363447363447
1.3451353451353452
1.3455343455343456
1.345933345933346
1.3463323463323462
1.3467313467313469
1.347130347130347
1.3475293475293475
1.347928347928348
1.3483273483273484
1.3487263487263488
1.3491253491253492
1.3495243495243496
1.3499233499233498
1.3503223503223503
1.35

1.5047355047355047
1.5051345051345053
1.5055335055335055
1.505932505932506
1.5063315063315064
1.5067305067305068
1.5071295071295072
1.5075285075285074
1.507927507927508
1.5083265083265083
1.5087255087255087
1.5091245091245091
1.5095235095235096
1.50992250992251
1.5103215103215102
1.5107205107205108
1.511119511119511
1.5115185115185117
1.511917511917512
1.5123165123165123
1.5127155127155127
1.5131145131145132
1.5135135135135136
1.5139125139125138
1.5143115143115145
1.5147105147105147
1.515109515109515
1.5155085155085155
1.515907515907516
1.5163065163065164
1.5167055167055166
1.5171045171045172
1.5175035175035174
1.517902517902518
1.5183015183015183
1.5187005187005187
1.5190995190995191
1.5194985194985196
1.51989751989752
1.5202965202965202
1.5206955206955208
1.521094521094521
1.5214935214935215
1.5218925218925219
1.5222915222915223
1.5226905226905227
1.523089523089523
1.5234885234885236
1.5238875238875238
1.5242865242865244
1.5246855246855247
1.525084525084525
1.5254835254835255
1.52588

1.6790986790986793
1.6794976794976795
1.67989667989668
1.6802956802956803
1.6806946806946808
1.6810936810936812
1.6814926814926814
1.681891681891682
1.6822906822906822
1.6826896826896827
1.683088683088683
1.6834876834876835
1.683886683886684
1.6842856842856841
1.6846846846846848
1.685083685083685
1.6854826854826856
1.6858816858816859
1.6862806862806863
1.6866796866796867
1.6870786870786871
1.6874776874776876
1.6878766878766878
1.6882756882756884
1.6886746886746886
1.689073689073689
1.6894726894726895
1.68987168987169
1.6902706902706903
1.6906696906696905
1.6910686910686912
1.6914676914676914
1.691866691866692
1.6922656922656922
1.6926646926646927
1.693063693063693
1.6934626934626935
1.693861693861694
1.6942606942606941
1.6946596946596948
1.695058695058695
1.6954576954576956
1.6958566958566959
1.6962556962556963
1.6966546966546967
1.6970536970536971
1.6974526974526976
1.6978516978516978
1.6982506982506984
1.6986496986496986
1.699048699048699
1.6994476994476995
1.6998466998467
1.70024570

...

10.542892542892544
10.543291543291543
10.543690543690545
10.544089544089545
10.544488544488544
10.544887544887546
10.545286545286546
10.545685545685545
10.546084546084547
10.546483546483547
10.546882546882546
10.547281547281548
10.547680547680548
10.548079548079548
10.548478548478549
10.548877548877549
10.54927654927655
10.54967554967555
10.55007455007455
10.550473550473551
10.550872550872551
10.551271551271551
10.551670551670552
10.552069552069552
10.552468552468552
10.552867552867554
10.553266553266553
10.553665553665555
10.554064554064555
10.554463554463554
10.554862554862556
10.555261555261556
10.555660555660555
10.556059556059557
10.556458556458557
10.556857556857556
10.557256557256558
10.557655557655558
10.558054558054558
10.558453558453559
10.558852558852559
10.55925155925156
10.55965055965056
10.56004956004956
10.560448560448561
10.560847560847561
10.561246561246561
10.561645561645562
10.562044562044562
10.562443562443562
10.562842562842564
10.563241563241563
10.563640563640565

10.717654717654717
10.718053718053719
10.718452718452719
10.718851718851719
10.71925071925072
10.71964971964972
10.72004872004872
10.720447720447721
10.72084672084672
10.721245721245722
10.721644721644722
10.722043722043722
10.722442722442723
10.722841722841723
10.723240723240723
10.723639723639725
10.724038724038724
10.724437724437724
10.724836724836726
10.725235725235725
10.725634725634725
10.726033726033727
10.726432726432726
10.726831726831728
10.727230727230728
10.727629727629727
10.728028728028729
10.728427728427729
10.728826728826728
10.72922572922573
10.72962472962473
10.73002373002373
10.730422730422731
10.73082173082173
10.731220731220732
10.731619731619732
10.732018732018732
10.732417732417733
10.732816732816733
10.733215733215733
10.733614733614735
10.734013734013734
10.734412734412734
10.734811734811736
10.735210735210735
10.735609735609735
10.736008736008737
10.736407736407736
10.736806736806738
10.737205737205738
10.737604737604737
10.738003738003739
10.738402738402739
1

10.893214893214893
10.893613893613894
10.894012894012894
10.894411894411896
10.894810894810895
10.895209895209895
10.895608895608897
10.896007896007896
10.896406896406896
10.896805896805898
10.897204897204897
10.897603897603897
10.898002898002899
10.898401898401898
10.8988008988009
10.8991998991999
10.8995988995989
10.899997899997901
10.9003969003969
10.9007959007959
10.901194901194902
10.901593901593902
10.901992901992902
10.902391902391903
10.902790902790903
10.903189903189903
10.903588903588904
10.903987903987904
10.904386904386905
10.904785904785905
10.905184905184905
10.905583905583907
10.905982905982906
10.906381906381906
10.906780906780908
10.907179907179907
10.907578907578907
10.907977907977909
10.908376908376908
10.90877590877591
10.90917490917491
10.90957390957391
10.909972909972911
10.91037191037191
10.91077091077091
10.911169911169912
10.911568911568912
10.911967911967912
10.912366912366913
10.912765912765913
10.913164913164913
10.913563913563914
10.913962913962914
10.91436

11.067977067977068
11.068376068376068
11.06877506877507
11.06917406917407
11.06957306957307
11.06997206997207
11.07037107037107
11.07077007077007
11.071169071169072
11.071568071568072
11.071967071967073
11.072366072366073
11.072765072765073
11.073164073164074
11.073563073563074
11.073962073962074
11.074361074361075
11.074760074760075
11.075159075159075
11.075558075558076
11.075957075957076
11.076356076356078
11.076755076755077
11.077154077154077
11.077553077553079
11.077952077952078
11.078351078351078
11.07875007875008
11.07914907914908
11.07954807954808
11.07994707994708
11.08034608034608
11.08074508074508
11.081144081144082
11.081543081543082
11.081942081942083
11.082341082341083
11.082740082740083
11.083139083139084
11.083538083538084
11.083937083937084
11.084336084336085
11.084735084735085
11.085134085134085
11.085533085533086
11.085932085932086
11.086331086331086
11.086730086730087
11.087129087129087
11.087528087528089
11.087927087927088
11.088326088326088
11.08872508872509
11.089

11.243537243537244
11.243936243936245
11.244335244335245
11.244734244734245
11.245133245133246
11.245532245532246
11.245931245931246
11.246330246330247
11.246729246729247
11.247128247128247
11.247527247527248
11.247926247926248
11.248325248325248
11.24872424872425
11.24912324912325
11.24952224952225
11.24992124992125
11.25032025032025
11.250719250719252
11.251118251118251
11.251517251517251
11.251916251916253
11.252315252315253
11.252714252714252
11.253113253113254
11.253512253512254
11.253911253911253
11.254310254310255
11.254709254709255
11.255108255108256
11.255507255507256
11.255906255906256
11.256305256305257
11.256704256704257
11.257103257103257
11.257502257502258
11.257901257901258
11.258300258300258
11.25869925869926
11.25909825909826
11.25949725949726
11.25989625989626
11.26029526029526
11.260694260694262
11.261093261093261
11.261492261492261
11.261891261891263
11.262290262290263
11.262689262689262
11.263088263088264
11.263487263487264
11.263886263886263
11.264285264285265
11.